# Lesses More

Writing a naive for loop to find out the behaviour of $f(a,b,c,d)$ reveals a few facts to us (the loop takes 3 minutes to run):

1. $a = 0$
2. $2a \leq b$, $2b \leq c$, $2c \leq d$
3. Each successive $(a,b,c,d)$ is larger than the preceding one, element-wise
4. For each $M$, there is a unique quadruplet (up to permutations) that has the minimal f value   

In [3]:
def f(a: int, b: int, c: int, d: int) -> int:
    if (a,b,c,d) == (0,0,0,0):
        return 1

    return 1 + f(abs(a - b), abs(b - c), abs(c - d), abs(d - a))

M: int = 0
sum_of_values: int = 40_000_000

# for a in range(0, 101):
#     for b in range(0, 101):
#         for c in range(0, 101):
#             for d in range(0, 101):
#                 curr_f_value = f(a,b,c,d)
#                 if curr_f_value > M or (curr_f_value == M and a + b + c + d < sum_of_values):
#                     M = curr_f_value
#                     sum_of_values = a + b + c + d
#                     print("M:", M, "a:", a, "b:", b, "c:", c, "d:", d) 

Using these observations, we can speed up the search for quadruplets, and divert some computational resources to inspecting the way in which each quadruplet unfolds, which reveals more facts:

1. The second level from the bottom is always a power of 2
2. The third level from the bottom to the second level from the top are all subsequences of tribonacci sequences of (0, 0, 1), (0, 1, 0) and (1, 1, 1) repeating.
3. The top quadruplet can be composed as follows:
   1. Let $s$ be the top quadruplet
   2. Let $t$ be the second-from-top quadruplet
   3. Then $s = (0, t[0], t[1] + s[1], t[2] + s[2])$

In [4]:
def expand(a: int, b: int, c: int, d: int) -> None:
    print(a, b, c, d)

    if (a,b,c,d) == (0,0,0,0):
        return

    expand(abs(a - b), abs(b - c), abs(c - d), abs(d - a))

M = 0
sum_of_values = 40_000_000

for b in range(0, 151):
    for c in range(b * 2, 151):
        for d in range(c * 2, 151):
            curr_f_value = f(0,b,c,d)
            if curr_f_value > M or (curr_f_value == M and b + c + d < sum_of_values):
                M = curr_f_value
                sum_of_values = b + c + d
                expand(0, b, c, d)
                print()

0 0 0 0

0 0 0 1
0 0 1 1
0 1 0 1
1 1 1 1
0 0 0 0

0 0 1 3
0 1 2 3
1 1 1 3
0 0 2 2
0 2 0 2
2 2 2 2
0 0 0 0

0 1 2 4
1 1 2 4
0 1 2 3
1 1 1 3
0 0 2 2
0 2 0 2
2 2 2 2
0 0 0 0

0 1 4 9
1 3 5 9
2 2 4 8
0 2 4 6
2 2 2 6
0 0 4 4
0 4 0 4
4 4 4 4
0 0 0 0

0 2 5 11
2 3 6 11
1 3 5 9
2 2 4 8
0 2 4 6
2 2 2 6
0 0 4 4
0 4 0 4
4 4 4 4
0 0 0 0

0 2 6 13
2 4 7 13
2 3 6 11
1 3 5 9
2 2 4 8
0 2 4 6
2 2 2 6
0 0 4 4
0 4 0 4
4 4 4 4
0 0 0 0

0 5 14 31
5 9 17 31
4 8 14 26
4 6 12 22
2 6 10 18
4 4 8 16
0 4 8 12
4 4 4 12
0 0 8 8
0 8 0 8
8 8 8 8
0 0 0 0

0 6 17 37
6 11 20 37
5 9 17 31
4 8 14 26
4 6 12 22
2 6 10 18
4 4 8 16
0 4 8 12
4 4 4 12
0 0 8 8
0 8 0 8
8 8 8 8
0 0 0 0

0 7 20 44
7 13 24 44
6 11 20 37
5 9 17 31
4 8 14 26
4 6 12 22
2 6 10 18
4 4 8 16
0 4 8 12
4 4 4 12
0 0 8 8
0 8 0 8
8 8 8 8
0 0 0 0

0 17 48 105
17 31 57 105
14 26 48 88
12 22 40 74
10 18 34 62
8 16 28 52
8 12 24 44
4 12 20 36
8 8 16 32
0 8 16 24
8 8 8 24
0 0 16 16
0 16 0 16
16 16 16 16
0 0 0 0

0 20 57 125
20 37 68 125
17 31 57 105
14 26 48 88
12 

See `test.txt` for the pattern in the second-from-top quadruplet. From the third optimal quadruplet onwards, the second-from-top quadruplet is always drawn from a rotation of the three tribonacci sequences, with index offset + 2. This reveals to us a quick way of generating the top quadruplet.

In [5]:
zero_zero_one = [0, 0, 1, 1, 2, 4, 7, 13, 24, 44, 81, 149, 274, 504, 927, 1705, 3136, 5768, 10609, 19513, 35890, 66012, 121415, 223317, 410744, 755476, 1389537, 2555757, 4700770, 8646064, 15902591, 29249425, 53798080, 98950096, 181997601, 334745777, 615693474, 1132436852 
]

zero_one_zero = [0, 1, 0, 1, 2, 3, 6, 11, 20, 37, 68, 125, 230, 423, 778, 1431, 2632, 4841, 8904, 16377, 30122, 55403, 101902, 187427, 344732, 634061, 1166220, 2145013, 3945294, 7256527, 13346834, 24548655, 45152016, 83047505, 152748176, 280947697, 516743378, 950439251
]

one_one_one = [1, 1, 1, 3, 5, 9, 17, 31, 57, 105, 193, 355, 653, 1201, 2209, 4063, 7473, 13745, 25281, 46499, 85525, 157305, 289329, 532159, 978793, 1800281, 3311233, 6090307, 11201821, 20603361, 37895489, 69700671, 128199521, 235795681, 433695873, 797691075, 1467182629]

sequences = [zero_one_zero, zero_zero_one, one_one_one]

i = 0 # This is to index sequences
j = 2 # This is to index the sequence in question

while 1:
    a, b, c, d = sequences[i][j:j+4]
    top = [0] * 4
    top[1] = a
    top[2] = b + top[1]
    top[3] = c + top[2]

    if top[3] > 10_000_000:
        break
    else:
        print(top)
        i = (i + 1) % 3
        if i == 0:
            j += 2

[0, 0, 1, 3]
[0, 1, 2, 4]
[0, 1, 4, 9]
[0, 2, 5, 11]
[0, 2, 6, 13]
[0, 5, 14, 31]
[0, 6, 17, 37]
[0, 7, 20, 44]
[0, 17, 48, 105]
[0, 20, 57, 125]
[0, 24, 68, 149]
[0, 57, 162, 355]
[0, 68, 193, 423]
[0, 81, 230, 504]
[0, 193, 548, 1201]
[0, 230, 653, 1431]
[0, 274, 778, 1705]
[0, 653, 1854, 4063]
[0, 778, 2209, 4841]
[0, 927, 2632, 5768]
[0, 2209, 6272, 13745]
[0, 2632, 7473, 16377]
[0, 3136, 8904, 19513]
[0, 7473, 21218, 46499]
[0, 8904, 25281, 55403]
[0, 10609, 30122, 66012]
[0, 25281, 71780, 157305]
[0, 30122, 85525, 187427]
[0, 35890, 101902, 223317]
[0, 85525, 242830, 532159]
[0, 101902, 289329, 634061]
[0, 121415, 344732, 755476]
[0, 289329, 821488, 1800281]
[0, 344732, 978793, 2145013]
[0, 410744, 1166220, 2555757]
[0, 978793, 2779074, 6090307]
[0, 1166220, 3311233, 7256527]
[0, 1389537, 3945294, 8646064]
